In [95]:
import requests

In [96]:
api_url = 'https://dhm.agiv.be/api/elevation/v1'

In [97]:
test_url = 'dataset'
response = requests.get(api_url)

In [160]:
post_url = "https://dhm.agiv.be/api/elevation/v1/DHMV2/request"

In [188]:
json = {
    "Samples": 30,
    "SrsIn": 31370,
    "SrsOut": 31370,
    "LineString": {}
}

In [196]:
xml = f"""
<Query xmlns:xsd="http://www.w3.org/2001/XMLSchema" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance">
  <Samples>30</Samples>
  <SrsIn>31370</SrsIn>
  <SrsOut>31370</SrsOut>
  <LineString xmlns="http://www.opengis.net/gml">
    <posList>133630.81 200349.78 133631.16 200461.03 133771.21 200460.59</posList>
  </LineString>
</Query>
"""

In [197]:
headers = {'Content-type': 'application/xml'}

In [198]:
response = requests.post(post_url, xml, headers=headers)

In [199]:
response.json()

[[0.0, 133630.81, 200349.78, 10.55],
 [8.66556006001514, 133630.8372623012, 200358.4455171756, 10.9],
 [17.33112012003028, 133630.86452460245, 200367.11103435123, 11.14],
 [25.996680180045423, 133630.89178690367, 200375.77655152683, 11.35],
 [34.66224024006056, 133630.9190492049, 200384.4420687024, 11.81],
 [43.3278003000757, 133630.94631150615, 200393.107585878, 12.02],
 [51.99336036009084, 133630.97357380734, 200401.77310305362, 12.09],
 [60.65892042010598, 133631.00083610858, 200410.4386202292, 12.06],
 [69.32448048012112, 133631.02809840982, 200419.10413740482, 12.14],
 [77.99004054013626, 133631.05536071103, 200427.76965458042, 12.75],
 [86.6556006001514, 133631.08262301228, 200436.43517175602, 12.76],
 [95.32116066016654, 133631.1098853135, 200445.10068893165, 12.84],
 [103.98672072018168, 133631.1371476147, 200453.76620610722, 13.02],
 [112.65228078019682, 133632.56172330194, 200461.0255961567, 13.14],
 [121.31784084021196, 133641.22724059556, 200460.9983713969, 13.1],
 [129.983

In [ ]:
# use a formatted string to get the result

xml = f"""
<Query xmlns:xsd="http://www.w3.org/2001/XMLSchema" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance">
  <Samples>30</Samples>
  <SrsIn>31370</SrsIn>
  <SrsOut>31370</SrsOut>
  <LineString xmlns="http://www.opengis.net/gml">
    <posList>{x} {y}</posList>
  </LineString>
</Query>
"""

In [201]:
# Would you look at that!
# The logic would be to do it like this: 

"""
1-1-1-1-1-1-1-1
2-2-2-2-2-2-2-2
3-3-3-3-3-3-3-3
4-4-4-4-4-4-4-4
5-5-5-5-5-5-5-5
6-6-6-6-6-6-6-6
"""

# each line represents a request made with the most left and the most right coordinates as input.
# each request uses n samples that correspond with the amount of measurements of each line.
# each new request forms a new line, which can be stored in a numpy array.
# This numpy array is used to plot a surface map.

# gather coordinates (x,y)
house_crds = (155151.12, 211718.34) 

# calculate cropped plot size 
house_crop_size = [house_crds[0]-100, house_crds[0]+100, house_crds[1]-100, house_crds[1]+100]

for x in range(200):
    

[154992.61, 155192.61, 211642.822, 211842.822]